# 🧼 Virginia Food Facility Inspection Data Prep
This notebook merges and cleans the 2022–2025 violation datasets.

In [1]:
import os
import pandas as pd
from pathlib import Path

data_folder = Path("data")
years = [2022, 2023, 2024, 2025]

In [2]:
# If preprocessed file exists, just load it
if os.path.exists("cleaned_inspections.parquet"):
    merged_df = pd.read_parquet("cleaned_inspections.parquet")
    print("Loaded cached Parquet file.")
else:
    # Load datasets for each year
    dfs = []
    for year in years:
        file = data_folder / f"Food_Facility_Inspection_Violations_{year if year < 2025 else '2025toDate'}.xlsx"
        df = pd.read_excel(file)
        df["Year"] = year
        dfs.append(df)

    # Merge all dataframes into one
    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.columns = merged_df.columns.str.strip()

    # Save for future reuse
    merged_df.to_parquet("cleaned_inspections.parquet", index=False)
    print("Created and saved merged Parquet file.")

Loaded cached Parquet file.


In [3]:
# Standardize column names
merged_df['InspectionDate'] = pd.to_datetime(merged_df['InspectionDate'], errors='coerce')
merged_df['facilityRiskRating'] = pd.to_numeric(merged_df['facilityRiskRating'], errors='coerce')

# Filter only currently permitted facilities
merged_df = merged_df[merged_df['status'].str.lower() == 'permitted']


In [4]:
# Add derived column flags
merged_df['isRepeat'] = merged_df['violationType'].fillna('').str.contains('R')
merged_df['isCorrected'] = merged_df['violationType'].fillna('').str.contains('COS')
merged_df['isPriority'] = merged_df['class'].str.strip().str.lower() == 'priority'

merged_df['permitType'] = merged_df['permitType'].str.strip().str.title()
merged_df['City'] = merged_df['City'].str.strip().str.title()
merged_df['Zip'] = merged_df['Zip'].astype(str).str[:5]
merged_df['Year'] = merged_df['InspectionDate'].dt.year

merged_df.sort_values(by='InspectionDate', inplace=True)

In [5]:
# Check for missing values in key columns
merged_df[['InspectionDate', 'violationType', 'permitType']].isnull().sum()


InspectionDate    0
violationType     0
permitType        0
dtype: int64

In [6]:
# Quick Summary
print("Top Violation Types:")
print(merged_df['violationType'].value_counts().head(10))

print("\nTop ZIP Codes:")
print(merged_df['Zip'].value_counts().head(10))

print("\nTop Permit Types:")
print(merged_df['permitType'].value_counts().head(10))


Top Violation Types:
violationType
Violation               119771
COS                      77756
V                        36431
COS,Violation            27112
COS,V                    14520
Repeat                    9447
Repeat,Violation          8126
COS,Repeat,Violation      3466
COS,Repeat                2919
R,V                       2583
Name: count, dtype: int64

Top ZIP Codes:
Zip
23320    7712
23233    4823
23502    4638
23451    4412
23294    4299
22314    3952
23230    3808
20147    3734
22401    3580
24060    3569
Name: count, dtype: int64

Top Permit Types:
permitType
Full Service Restaurant              170063
Fast Food                             81220
Educational Facility Food Service     11132
Carry Out                              9608
Adult Care Home Service                6387
Mobile Food Unit                       5613
Child Care Food Service                5563
Health Care Food Facility              4600
Continental Breakfast                  3940
Convenience Store

In [7]:
# Convert all object-type columns to string
for col in merged_df.select_dtypes(include='object').columns:
    merged_df[col] = merged_df[col].astype(str)

In [8]:
# Save as CSV (no issue here)
merged_df.to_csv("cleaned_inspections.csv", index=False)

# Fix all object columns for Parquet export
for col in merged_df.select_dtypes(include='object').columns:
    merged_df[col] = merged_df[col].astype(str)

# Save as Parquet
merged_df.to_parquet("cleaned_inspections.parquet", index=False)

In [9]:
%pip install nbformat

import plotly.express as px

# Ensure ZIP is treated as string for categorical plotting
merged_df['Zip'] = merged_df['Zip'].astype(str)

# Get top 10 ZIP codes by violation count
top_zips = merged_df['Zip'].value_counts().nlargest(10).sort_values(ascending=True)

# Create horizontal bar chart
fig = px.bar(
    x=top_zips.values,
    y=top_zips.index,
    orientation='h',
    labels={'x': 'Violations', 'y': 'ZIP Code'},
    title='Top 10 ZIP Codes by Violation Count'
)

fig.update_layout(
    yaxis_title='ZIP Code',
    xaxis_title='Violations',
    title_x=0.5,
    height=500,
    bargap=0.3
)

fig.show()

Note: you may need to restart the kernel to use updated packages.
